In [1]:
import pandas as pd
import numpy as np
import re
import  collections
from statistics import StatisticsError

In [2]:
# https://stackoverflow.com/questions/33511259/finding-majority-votes-on-1s-1s-and-0s-in-list-python
def _counts(data):
    # Generate a table of sorted (value, frequency) pairs.
    table = collections.Counter(iter(data)).most_common()
    if not table:
        return table
    # Extract the values with the highest frequency.
    maxfreq = table[0][1]
    for i in range(1, len(table)):
        if table[i][1] != maxfreq:
            table = table[:i]
            break
    return table

def mode(data):
    table = _counts(data)
    if len(table) == 1:
        return table[0][0]
    elif table:
        list_table = [table[0][0], table[1][0], table[2][0]]
        list_table.sort()
        delta_0_1 = abs(list_table[0] - list_table[1])
        delta_1_2 = abs(list_table[1] - list_table[2])
        delta_0_2 = abs(list_table[0] - list_table[2])
        if delta_0_2 < 5:
            return sum(list_table)/len(list_table)
        else:
            if delta_0_1 < 5:
                if delta_0_1 < delta_1_2:
                    return (list_table[0] + list_table[1]) / 2.0
                else:
                    return (list_table[1] + list_table[2]) / 2.0
            else:
                if delta_1_2 < 5:
                    return (list_table[1] + list_table[2]) / 2.0
                else:
                    return table[1][0]
        
#         raise StatisticsError(
#                 'no unique mode; found %d equally common values' % len(table)
#                 )
#     else:
#         raise StatisticsError('no mode for empty data')

In [3]:
def get_common_data(file1, file2, file3):
    f1 = pd.read_csv(file1)
    f2 = pd.read_csv(file2)
    f3 = pd.read_csv(file3)

    # Get most common x
    data_x = pd.concat([f1.x, f2.x, f3.x] , axis=1)
    data_x.columns = ['sift_x', 'resnet_x', 'vgg_x']
    table_x=[]
    common_x = []
    for i in range(1200):
        table_x = [data_x.sift_x[i], data_x.resnet_x[i], data_x.vgg_x[i]]
        common_x.append(mode(table_x))
    common_x = pd.DataFrame(common_x)
    
    # Get most common y
    data_y = pd.concat([f1.y, f2.y, f3.y] , axis=1)
    data_y.columns = ['sift_y', 'resnet_y', 'vgg_y']
    table_y=[]
    common_y = []
    for i in range(1200):
        table_y = [data_y.sift_y[i], data_y.resnet_y[i], data_y.vgg_y[i]]
        common_y.append(mode(table_y))
    common_y = pd.DataFrame(common_y)
    
    # vote table
    data = pd.concat([f1.id, common_x, common_y] , axis=1)
    data.columns = ['id', 'x', 'y']
    return data
    
data = get_common_data('resnet_fix.csv', 'resnet.csv', 'vgg.csv')
data

,id,x,y
0,IMG4287_3,28.119322,-73.017280
1,IMG4288_5,89.052655,-37.017280
2,IMG4289_5,78.419322,-41.817280
3,IMG4290_4,-35.080678,118.482720
4,IMG4291_5,-91.830678,130.049386
...,...,...,...
1195,IMG5482_1,-87.980678,104.582720
1196,IMG5483_2,59.119322,-15.717280
1197,IMG5484_4,-115.280678,54.682720
1198,IMG5485_3,73.619322,-35.517280


In [4]:
data.to_csv('vote_final.csv', index = False)

In [5]:
# f1 = pd.read_csv('sift.csv')
# f2 = pd.read_csv('resnet.csv')
# f3 = pd.read_csv('vgg.csv')

# data_id = pd.concat([f1.id, f2.id, f3.id] , axis=1)
# data_id.columns = ['sift_id', 'resnet_id', 'vgg_id']
# data_id

# def function(a, b):
#     if a == b:
#         return 1
#     else:
#         return 0

# data_id['bool'] = data_id.apply(lambda x : function(x.resnet_id,x.vgg_id),axis = 1)
# list(data_id['bool'].head(1200))

In [6]:
resnet = pd.read_csv('resnet.csv')
vote = pd.read_csv('vote.csv')
combine =  pd.concat([resnet.id, resnet.x, resnet.y, vote.x, vote.y] , axis=1)
combine.columns = ['id', 'resnet_x', 'resnet_y', 'vote_x', 'vote_y']

mae = 0
for i in range(1200):
    ae = abs(combine.vote_x[i] - combine.resnet_x[i]) + abs(combine.vote_y[i] - combine.resnet_y[i])
    mae += abs(ae)
mae = mae/1200
print(mae)

3.437208333480001


In [7]:
resnet = pd.read_csv('vgg_fix.csv')
vote = pd.read_csv('vote.csv')
combine =  pd.concat([resnet.id, resnet.x, resnet.y, vote.x, vote.y] , axis=1)
combine.columns = ['id', 'resnet_x', 'resnet_y', 'vote_x', 'vote_y']
combine

mae = 0
for i in range(1200):
    ae = abs(combine.vote_x[i] - combine.resnet_x[i]) + abs(combine.vote_y[i] - combine.resnet_y[i])
    mae += abs(ae)
mae = mae/1200
print(mae)

13.143041666530843


In [8]:
resnet = pd.read_csv('resnet_fix.csv')
vote = pd.read_csv('vote.csv')
combine =  pd.concat([resnet.id, resnet.x, resnet.y, vote.x, vote.y] , axis=1)
combine.columns = ['id', 'resnet_x', 'resnet_y', 'vote_x', 'vote_y']
combine

mae = 0
for i in range(1200):
    ae = abs(combine.vote_x[i] - combine.resnet_x[i]) + abs(combine.vote_y[i] - combine.resnet_y[i])
    mae += abs(ae)
mae = mae/1200
print(mae)

10.116347222313902


In [9]:
combine.head(1200)

,id,resnet_x,resnet_y,vote_x,vote_y
0,IMG4287_3,29.819322,-73.41728,28.119322,-73.017280
1,IMG4288_5,91.219322,-38.91728,89.052655,-37.017280
2,IMG4289_5,78.419322,-41.81728,78.419322,-41.817280
3,IMG4290_4,-111.880678,120.08272,-35.080678,27.082720
4,IMG4291_5,-87.080678,128.48272,-91.830678,130.049386
...,...,...,...,...,...
1195,IMG5482_1,-82.180678,103.98272,-87.980678,104.582720
1196,IMG5483_2,56.719322,-16.91728,59.119322,-15.717280
1197,IMG5484_4,-115.280678,54.68272,-115.280678,54.682720
1198,IMG5485_3,73.619322,-35.51728,74.252655,-24.067280
